In [2]:
# make predictions on upcoming games


In [3]:
import pandas as pd

In [4]:
# read in upcoming games data
# write upcoming games to excel
upcoming_games = pd.read_excel(f'upcoming_nhl_games_2025.xlsx', header=0)
upcoming_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Game Link        114 non-null    object        
 1   Home_Team        114 non-null    object        
 2   Away_Team        114 non-null    object        
 3   Time             114 non-null    object        
 4   Date             114 non-null    datetime64[ns]
 5   Season           114 non-null    int64         
 6   Day_of_Week      114 non-null    object        
 7   Month            114 non-null    object        
 8   Game_Start_Hour  114 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 8.1+ KB


In [5]:
# # trim to games that are upcoming today only
# upcoming_games_trim = upcoming_games[upcoming_games['Date'] == pd.Timestamp.now().normalize()]

# trim to games that are upcoming tomorrow only
tomorrow = pd.Timestamp.now().normalize() + pd.Timedelta(days=1)
upcoming_games_trim = upcoming_games[upcoming_games['Date'] == tomorrow]

upcoming_games_trim

,Game Link,Home_Team,Away_Team,Time,Date,Season,Day_of_Week,Month,Game_Start_Hour
3,https://www.flashscore.com/match/hockey/florid...,Florida Panthers,Tampa Bay Lightning,17:00,2025-11-15,2025,Saturday,November,17
4,https://www.flashscore.com/match/hockey/anahei...,Minnesota Wild,Anaheim Ducks,18:00,2025-11-15,2025,Saturday,November,18
5,https://www.flashscore.com/match/hockey/caroli...,Carolina Hurricanes,Edmonton Oilers,19:00,2025-11-15,2025,Saturday,November,19
6,https://www.flashscore.com/match/hockey/chicag...,Chicago Blackhawks,Toronto Maple Leafs,19:00,2025-11-15,2025,Saturday,November,19
7,https://www.flashscore.com/match/hockey/columb...,Columbus Blue Jackets,New York Rangers,19:00,2025-11-15,2025,Saturday,November,19
8,https://www.flashscore.com/match/hockey/buffal...,Detroit Red Wings,Buffalo Sabres,19:00,2025-11-15,2025,Saturday,November,19
9,https://www.flashscore.com/match/hockey/boston...,Montreal Canadiens,Boston Bruins,19:00,2025-11-15,2025,Saturday,November,19
10,https://www.flashscore.com/match/hockey/los-an...,Ottawa Senators,Los Angeles Kings,19:00,2025-11-15,2025,Saturday,November,19
11,https://www.flashscore.com/match/hockey/new-je...,Washington Capitals,New Jersey Devils,19:00,2025-11-15,2025,Saturday,November,19
12,https://www.flashscore.com/match/hockey/dallas...,Dallas Stars,Philadelphia Flyers,20:00,2025-11-15,2025,Saturday,November,20


In [6]:
# read in modeling data
modeling_data = pd.read_excel(r'data/modeling_data.xlsx', header=0)

# filter to target season
target_season = upcoming_games_trim['Season'].iloc[0]
modeling_data = modeling_data[modeling_data['Season'] == target_season]

# drop cols that contain P1-P3 in the col name
modeling_data = modeling_data[modeling_data.columns.drop(list(modeling_data.filter(regex='P[1-3]')))]

# drop other cols
# list of drop cols that won't be used in modeling
drop_cols = ['Game_ID', 'Reg_Home_Win', 'Reg_Away_Win',
         'Conf_Pair', 'Team_Pair', 'Div_Pair', 'Conf_Matchup', 'Div_Matchup',
        'Odds_1', 'Odds_2', 'Odds_X', 'Start_Hour_Group', 'Season', 'Month', 'Day_of_Week',
        'Reg_Tie'
    ]

modeling_data = modeling_data.drop(columns=drop_cols)

# inspect
modeling_data.info()
modeling_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, 89 to 3404
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Date                              374 non-null    datetime64[ns]
 1   Home_Team                         374 non-null    object        
 2   Away_Team                         374 non-null    object        
 3   prop_Reg_Home_Win_Home            374 non-null    float64       
 4   prop_Reg_Away_Win_Home            374 non-null    float64       
 5   prop_Reg_Tie_Home                 374 non-null    float64       
 6   prop_reg_home_goal_diff_Home      374 non-null    float64       
 7   prop_reg_away_goal_diff_Home      374 non-null    float64       
 8   avg_reg_home_goals_per_game_Home  374 non-null    float64       
 9   avg_reg_away_goals_per_game_Home  374 non-null    float64       
 10  prop_Reg_Home_Win_Away            374 non-null    flo

,Date,Home_Team,Away_Team,prop_Reg_Home_Win_Home,prop_Reg_Away_Win_Home,prop_Reg_Tie_Home,prop_reg_home_goal_diff_Home,prop_reg_away_goal_diff_Home,avg_reg_home_goals_per_game_Home,avg_reg_away_goals_per_game_Home,prop_Reg_Home_Win_Away,prop_Reg_Away_Win_Away,prop_Reg_Tie_Away,prop_reg_home_goal_diff_Away,prop_reg_away_goal_diff_Away,avg_reg_home_goals_per_game_Away,avg_reg_away_goals_per_game_Away
89,2025-09-22,Anaheim Ducks,Utah Mammoth,1.000000,0.000000,0.0,0.857143,0.142857,6.00,1.00,1.000000,0.0,0.000000,0.750000,0.250000,4.500000,1.5
90,2025-09-24,Anaheim Ducks,Los Angeles Kings,0.500000,0.500000,0.0,0.600000,0.400000,3.00,2.00,0.000000,1.0,0.000000,0.142857,0.857143,0.500000,3.0
91,2025-09-29,Anaheim Ducks,San Jose Sharks,0.666667,0.333333,0.0,0.600000,0.400000,3.00,2.00,1.000000,0.0,0.000000,0.600000,0.400000,3.000000,2.0
92,2025-10-14,Anaheim Ducks,Pittsburgh Penguins,0.750000,0.250000,0.0,0.590909,0.409091,3.25,2.25,0.333333,0.5,0.166667,0.464286,0.535714,2.166667,2.5
93,2025-10-16,Anaheim Ducks,Carolina Hurricanes,0.600000,0.400000,0.0,0.518519,0.481481,2.80,2.60,0.200000,0.4,0.400000,0.406250,0.593750,2.600000,3.8


In [7]:
# store list of home teams and away teams in upcoming games
home_teams = upcoming_games_trim['Home_Team'].unique().tolist()
away_teams = upcoming_games_trim['Away_Team'].unique().tolist()

print(f'Home Teams: {home_teams}')
print(f'Away Teams: {away_teams}')

Home Teams: ['Florida Panthers', 'Minnesota Wild', 'Carolina Hurricanes', 'Chicago Blackhawks', 'Columbus Blue Jackets', 'Detroit Red Wings', 'Montreal Canadiens', 'Ottawa Senators', 'Washington Capitals', 'Dallas Stars', 'St. Louis Blues', 'Calgary Flames', 'Seattle Kraken']
Away Teams: ['Tampa Bay Lightning', 'Anaheim Ducks', 'Edmonton Oilers', 'Toronto Maple Leafs', 'New York Rangers', 'Buffalo Sabres', 'Boston Bruins', 'Los Angeles Kings', 'New Jersey Devils', 'Philadelphia Flyers', 'Vegas Golden Knights', 'Winnipeg Jets', 'San Jose Sharks']


In [8]:

# filter to teams in upcoming games
home_data = modeling_data[modeling_data['Home_Team'].isin(home_teams)]
away_data = modeling_data[modeling_data['Away_Team'].isin(away_teams)]

# get last row for each Home_Team
last_rows_home = home_data.groupby('Home_Team').tail(1).reset_index(drop=True)

# drop cols that end with _Away
last_rows_home = last_rows_home[last_rows_home.columns.drop(list(last_rows_home.filter(regex='_Away$')))]

# drop Away_Team
last_rows_home = last_rows_home.drop(columns=['Away_Team', 'Date'])
last_rows_home

,Home_Team,prop_Reg_Home_Win_Home,prop_Reg_Away_Win_Home,prop_Reg_Tie_Home,prop_reg_home_goal_diff_Home,prop_reg_away_goal_diff_Home,avg_reg_home_goals_per_game_Home,avg_reg_away_goals_per_game_Home
0,Calgary Flames,0.272727,0.545455,0.181818,0.431034,0.568966,2.272727,3.000000
1,Carolina Hurricanes,0.500000,0.400000,0.100000,0.529412,0.470588,3.600000,3.200000
2,Chicago Blackhawks,0.222222,0.555556,0.222222,0.452381,0.547619,2.111111,2.555556
3,Columbus Blue Jackets,0.500000,0.500000,0.000000,0.482143,0.517857,2.700000,2.900000
4,Dallas Stars,0.416667,0.250000,0.333333,0.492063,0.507937,2.583333,2.666667
5,Detroit Red Wings,0.500000,0.333333,0.166667,0.513514,0.486486,3.166667,3.000000
6,Florida Panthers,0.555556,0.111111,0.333333,0.640000,0.360000,3.555556,2.000000
7,Minnesota Wild,0.230769,0.384615,0.384615,0.435897,0.564103,2.615385,3.384615
8,Montreal Canadiens,0.250000,0.333333,0.416667,0.472973,0.527027,2.916667,3.250000
9,Ottawa Senators,0.333333,0.333333,0.333333,0.500000,0.500000,2.750000,2.750000


In [9]:

# get last row for each Away_Team
last_rows_away = away_data.groupby('Away_Team').tail(1).reset_index(drop=True)

# drop cols that end with _Home
last_rows_away = last_rows_away[last_rows_away.columns.drop(list(last_rows_away.filter(regex='_Home$')))]

# drop Home_Team
last_rows_away = last_rows_away.drop(columns=['Home_Team', 'Date'])

last_rows_away.info()
last_rows_away.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Away_Team                         13 non-null     object 
 1   prop_Reg_Home_Win_Away            13 non-null     float64
 2   prop_Reg_Away_Win_Away            13 non-null     float64
 3   prop_Reg_Tie_Away                 13 non-null     float64
 4   prop_reg_home_goal_diff_Away      13 non-null     float64
 5   prop_reg_away_goal_diff_Away      13 non-null     float64
 6   avg_reg_home_goals_per_game_Away  13 non-null     float64
 7   avg_reg_away_goals_per_game_Away  13 non-null     float64
dtypes: float64(7), object(1)
memory usage: 964.0+ bytes


,Away_Team,prop_Reg_Home_Win_Away,prop_Reg_Away_Win_Away,prop_Reg_Tie_Away,prop_reg_home_goal_diff_Away,prop_reg_away_goal_diff_Away,avg_reg_home_goals_per_game_Away,avg_reg_away_goals_per_game_Away
0,Toronto Maple Leafs,0.428571,0.428571,0.142857,0.491525,0.508475,4.142857,4.285714
1,Vegas Golden Knights,0.375000,0.250000,0.375000,0.580645,0.419355,2.250000,1.625000
2,New Jersey Devils,0.250000,0.625000,0.125000,0.446809,0.553191,2.625000,3.250000
3,Buffalo Sabres,0.500000,0.166667,0.333333,0.542857,0.457143,3.166667,2.666667
4,New York Rangers,0.222222,0.555556,0.222222,0.422222,0.577778,2.111111,2.888889


In [10]:
# left join home and away data on team names
upcoming_games_final = pd.merge(upcoming_games_trim, last_rows_home, on='Home_Team', how='left')
upcoming_games_final = pd.merge(upcoming_games_final, last_rows_away, on='Away_Team', how='left')

upcoming_games_final.info()
upcoming_games_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Game Link                         13 non-null     object        
 1   Home_Team                         13 non-null     object        
 2   Away_Team                         13 non-null     object        
 3   Time                              13 non-null     object        
 4   Date                              13 non-null     datetime64[ns]
 5   Season                            13 non-null     int64         
 6   Day_of_Week                       13 non-null     object        
 7   Month                             13 non-null     object        
 8   Game_Start_Hour                   13 non-null     int64         
 9   prop_Reg_Home_Win_Home            13 non-null     float64       
 10  prop_Reg_Away_Win_Home            13 non-null     fl

,Game Link,Home_Team,Away_Team,Time,Date,Season,Day_of_Week,Month,Game_Start_Hour,prop_Reg_Home_Win_Home,...,prop_reg_away_goal_diff_Home,avg_reg_home_goals_per_game_Home,avg_reg_away_goals_per_game_Home,prop_Reg_Home_Win_Away,prop_Reg_Away_Win_Away,prop_Reg_Tie_Away,prop_reg_home_goal_diff_Away,prop_reg_away_goal_diff_Away,avg_reg_home_goals_per_game_Away,avg_reg_away_goals_per_game_Away
0,https://www.flashscore.com/match/hockey/florid...,Florida Panthers,Tampa Bay Lightning,17:00,2025-11-15,2025,Saturday,November,17,0.555556,...,0.360000,3.555556,2.000000,0.200000,0.400000,0.400000,0.600000,0.400000,3.000000,2.000000
1,https://www.flashscore.com/match/hockey/anahei...,Minnesota Wild,Anaheim Ducks,18:00,2025-11-15,2025,Saturday,November,18,0.230769,...,0.564103,2.615385,3.384615,0.230769,0.384615,0.384615,0.462366,0.537634,3.307692,3.846154
2,https://www.flashscore.com/match/hockey/caroli...,Carolina Hurricanes,Edmonton Oilers,19:00,2025-11-15,2025,Saturday,November,19,0.500000,...,0.470588,3.600000,3.200000,0.000000,1.000000,0.000000,0.250000,0.750000,1.000000,3.000000
3,https://www.flashscore.com/match/hockey/chicag...,Chicago Blackhawks,Toronto Maple Leafs,19:00,2025-11-15,2025,Saturday,November,19,0.222222,...,0.547619,2.111111,2.555556,0.428571,0.428571,0.142857,0.491525,0.508475,4.142857,4.285714
4,https://www.flashscore.com/match/hockey/columb...,Columbus Blue Jackets,New York Rangers,19:00,2025-11-15,2025,Saturday,November,19,0.500000,...,0.517857,2.700000,2.900000,0.222222,0.555556,0.222222,0.422222,0.577778,2.111111,2.888889


In [11]:
# read in trained model
import joblib

basic_model = joblib.load('model/catboost_model_reg_tie.pkl')


In [12]:
# isolate features model needs for upcoming games
feature_cols = basic_model.feature_names_

# trim cols for data to predict
predict_df = upcoming_games_final[feature_cols]
predict_df.info()
predict_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Season                            13 non-null     int64  
 1   Month                             13 non-null     object 
 2   Day_of_Week                       13 non-null     object 
 3   prop_Reg_Home_Win_Home            13 non-null     float64
 4   prop_Reg_Away_Win_Home            13 non-null     float64
 5   prop_Reg_Tie_Home                 13 non-null     float64
 6   prop_reg_home_goal_diff_Home      13 non-null     float64
 7   prop_reg_away_goal_diff_Home      13 non-null     float64
 8   avg_reg_home_goals_per_game_Home  13 non-null     float64
 9   avg_reg_away_goals_per_game_Home  13 non-null     float64
 10  prop_Reg_Home_Win_Away            13 non-null     float64
 11  prop_Reg_Away_Win_Away            13 non-null     float64
 12  prop_Reg_T

,Season,Month,Day_of_Week,prop_Reg_Home_Win_Home,prop_Reg_Away_Win_Home,prop_Reg_Tie_Home,prop_reg_home_goal_diff_Home,prop_reg_away_goal_diff_Home,avg_reg_home_goals_per_game_Home,avg_reg_away_goals_per_game_Home,prop_Reg_Home_Win_Away,prop_Reg_Away_Win_Away,prop_Reg_Tie_Away,prop_reg_home_goal_diff_Away,prop_reg_away_goal_diff_Away,avg_reg_home_goals_per_game_Away,avg_reg_away_goals_per_game_Away
0,2025,November,Saturday,0.555556,0.111111,0.333333,0.640000,0.360000,3.555556,2.000000,0.200000,0.400000,0.400000,0.600000,0.400000,3.000000,2.000000
1,2025,November,Saturday,0.230769,0.384615,0.384615,0.435897,0.564103,2.615385,3.384615,0.230769,0.384615,0.384615,0.462366,0.537634,3.307692,3.846154
2,2025,November,Saturday,0.500000,0.400000,0.100000,0.529412,0.470588,3.600000,3.200000,0.000000,1.000000,0.000000,0.250000,0.750000,1.000000,3.000000
3,2025,November,Saturday,0.222222,0.555556,0.222222,0.452381,0.547619,2.111111,2.555556,0.428571,0.428571,0.142857,0.491525,0.508475,4.142857,4.285714
4,2025,November,Saturday,0.500000,0.500000,0.000000,0.482143,0.517857,2.700000,2.900000,0.222222,0.555556,0.222222,0.422222,0.577778,2.111111,2.888889


In [13]:
from catboost import CatBoostClassifier

# make predictions
# 5. Predict and evaluate proba
y_pred_proba = basic_model.predict_proba(predict_df)[:, 1]

# add preds toupcoming_games_final
upcoming_games_final['Pred_Tie_Proba'] = y_pred_proba

# inspect
inspect_cols = ['Date', 'Time', 'Home_Team', 'Away_Team', 'Pred_Tie_Proba'] + feature_cols
upcoming_games_final[inspect_cols].sort_values(by='Pred_Tie_Proba', ascending=False)


,Date,Time,Home_Team,Away_Team,Pred_Tie_Proba,Season,Month,Day_of_Week,prop_Reg_Home_Win_Home,prop_Reg_Away_Win_Home,...,prop_reg_away_goal_diff_Home,avg_reg_home_goals_per_game_Home,avg_reg_away_goals_per_game_Home,prop_Reg_Home_Win_Away,prop_Reg_Away_Win_Away,prop_Reg_Tie_Away,prop_reg_home_goal_diff_Away,prop_reg_away_goal_diff_Away,avg_reg_home_goals_per_game_Away,avg_reg_away_goals_per_game_Away
1,2025-11-15,18:00,Minnesota Wild,Anaheim Ducks,0.865770,2025,November,Saturday,0.230769,0.384615,...,0.564103,2.615385,3.384615,0.230769,0.384615,0.384615,0.462366,0.537634,3.307692,3.846154
0,2025-11-15,17:00,Florida Panthers,Tampa Bay Lightning,0.842222,2025,November,Saturday,0.555556,0.111111,...,0.360000,3.555556,2.000000,0.200000,0.400000,0.400000,0.600000,0.400000,3.000000,2.000000
10,2025-11-15,20:00,St. Louis Blues,Vegas Golden Knights,0.768224,2025,November,Saturday,0.307692,0.461538,...,0.559524,2.846154,3.615385,0.375000,0.250000,0.375000,0.580645,0.419355,2.250000,1.625000
9,2025-11-15,20:00,Dallas Stars,Philadelphia Flyers,0.747096,2025,November,Saturday,0.416667,0.250000,...,0.507937,2.583333,2.666667,0.666667,0.000000,0.333333,0.687500,0.312500,3.666667,1.666667
7,2025-11-15,19:00,Ottawa Senators,Los Angeles Kings,0.611053,2025,November,Saturday,0.333333,0.333333,...,0.500000,2.750000,2.750000,0.333333,0.500000,0.166667,0.433333,0.566667,2.166667,2.833333
5,2025-11-15,19:00,Detroit Red Wings,Buffalo Sabres,0.592863,2025,November,Saturday,0.500000,0.333333,...,0.486486,3.166667,3.000000,0.500000,0.166667,0.333333,0.542857,0.457143,3.166667,2.666667
6,2025-11-15,19:00,Montreal Canadiens,Boston Bruins,0.446616,2025,November,Saturday,0.250000,0.333333,...,0.527027,2.916667,3.250000,0.000000,0.750000,0.250000,0.391304,0.608696,2.250000,3.500000
11,2025-11-15,22:00,Calgary Flames,Winnipeg Jets,0.369542,2025,November,Saturday,0.272727,0.545455,...,0.568966,2.272727,3.000000,0.454545,0.363636,0.181818,0.525424,0.474576,2.818182,2.545455
3,2025-11-15,19:00,Chicago Blackhawks,Toronto Maple Leafs,0.362497,2025,November,Saturday,0.222222,0.555556,...,0.547619,2.111111,2.555556,0.428571,0.428571,0.142857,0.491525,0.508475,4.142857,4.285714
8,2025-11-15,19:00,Washington Capitals,New Jersey Devils,0.157102,2025,November,Saturday,0.454545,0.454545,...,0.465517,2.818182,2.454545,0.250000,0.625000,0.125000,0.446809,0.553191,2.625000,3.250000
